## Multi-Accent and Multi-Lingual Voice Clone Demo with MeloTTS

In [5]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

### Initialization

In this example, we will use the checkpoints from OpenVoiceV2. OpenVoiceV2 is trained with more aggressive augmentations and thus demonstrate better robustness in some cases.

In [6]:
ckpt_converter = 'checkpoints_v2/converter'
device = "cuda:0" if torch.cuda.is_available() else "cpu"
output_dir = 'outputs_v2'

tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

os.makedirs(output_dir, exist_ok=True)

c:\Users\ddcdi\Documents\code\open-voice\.venv\lib\site-packages\torch\nn\utils\weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


Loaded checkpoint 'checkpoints_v2/converter/checkpoint.pth'
missing/unexpected keys: [] []


### Obtain Tone Color Embedding
We only extract the tone color embedding for the target speaker. The source tone color embeddings can be directly loaded from `checkpoints_v2/ses` folder.

In [7]:
reference_speaker = './resources/c-div-seagull.wav' # This is the voice you want to clone
# reference_speaker = './resources/example_reference.mp3' # This is the voice you want to clone
target_se, audio_name = se_extractor.get_se(reference_speaker, tone_color_converter, vad=False)

OpenVoice version: v2


#### Use MeloTTS as Base Speakers

MeloTTS is a high-quality multi-lingual text-to-speech library by @MyShell.ai, supporting languages including English (American, British, Indian, Australian, Default), Spanish, French, Chinese, Japanese, Korean. In the following example, we will use the models in MeloTTS as the base speakers. 

In [8]:
from melo.api import TTS

texts = {
    'EN': "Hello!. Welcome to Nottingham Trent University’s Open Day. I am a work in progress for the Real-time Open-day Bot for Information and Navigation, but you can call me ROBIN for short. I am an artificially intelligent chatbot and I am here to help you! I know all sorts of information about Nottingham, Nottingham Trent University, and the Department of Computer Science. Ask me any question and I will do my best to help you.",  # The newest English base speaker model
}


src_path = f'{output_dir}/clean-seagull-temp.wav'

# Speed is adjustable
speed = 1

model = TTS(language='EN', device=device)


source_se = torch.load(f'checkpoints_v2/base_speakers/ses/en-br.pth', map_location=device)
model.tts_to_file("HHello direct test", 1, src_path, speed=speed)
save_path = f'{output_dir}/one-off.wav'

# Run the tone color converter
encode_message = "@MyShell"
tone_color_converter.convert(
    audio_src_path=src_path, 
    src_se=source_se, 
    tgt_se=target_se, 
    output_path=save_path,
    message=encode_message)

 > Text split to sentences.
HHello direct test
 > ===========================


100%|██████████| 1/1 [00:00<00:00,  3.21it/s]


Audio too short, fail to add watermark
